# Tests de Word2Vec avec des vecteurs [NLPL](http://vectors.nlpl.eu/repository/)

## Français (NLPL/43)
Corpus :
- French CoNLL17 corpus
    - français, minuscules, sans lemmes, sans tags
    - 5494845765 tokens

Créé par Andrey Kutuzov (andreku@ifi.uio.no)

Paramètres :
- skipgram
- 100 dimensions
- fenêtre 10 mots
- taille du vocabulaire : 2567698 mots 

In [2]:
import os,gensim
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

data_path= "/votre/chemin/vers/nlpl/" # commentez les autres chemins
data_path="/Users/gilles/Downloads/NLPL/"

In [3]:
path_to_modelFr = os.path.join(data_path,'43/model.bin')
#Load W2V model. This will take some time.
modelFr = KeyedVectors.load_word2vec_format(path_to_modelFr, binary=True)
print('done loading Word2Vec')

done loading Word2Vec


In [4]:
import re
lettres=re.compile(r"^[^\W\d]+$",re.U)

def most_similar(mot="",positive=[],negative=[],topn=10,model=modelFr):
    result=[]
    if mot!="":
        similaires=model.most_similar(mot,topn=2*topn)
    elif positive:
        similaires=model.most_similar(positive=positive,negative=negative,topn=2*topn)
    else:
        print ("pb de paramêtres")
    for similaire in similaires:
        m=lettres.match(similaire[0])
        if m:
            result.append(similaire)
        if len(result)==topn:
            break
    return result


In [7]:
mots=[]
nonMots=[]
for k in modelFr.index_to_key:
    m=lettres.match(k)
    if m:
        mots.append(k)
    else:
        nonMots.append(k)
len(mots),len(nonMots)

(1523344, 1044354)

In [72]:
most_similar("homme")

[('femme', 0.7569620609283447),
 ('hommeet', 0.7099940776824951),
 ('hommme', 0.695120096206665),
 ('hommeen', 0.6883971691131592),
 ('hommea', 0.6845988035202026),
 ('jogpant', 0.6799124479293823),
 ('kwazi', 0.6795517206192017),
 ('hommepar', 0.6776106357574463),
 ('lionfemme', 0.6714594960212708),
 ('hommehomme', 0.6713407635688782)]

In [73]:
most_similar(positive=["roi","femme"],negative=["homme"],topn=5)

[('reine', 0.788600742816925),
 ('régente', 0.7626444697380066),
 ('waccho', 0.7577703595161438),
 ('rigonde', 0.7384146451950073),
 ('clodoswinthe', 0.7330183386802673)]

In [8]:
most_similar(positive=["cyclisme","maçon"],negative=["cycliste"],topn=5)

[('maconnerie', 0.6592905521392822),
 ('plâtrier', 0.6461008191108704),
 ('plaquiste', 0.6455990672111511),
 ('briqueteur', 0.6410598754882812),
 ('coffreur', 0.6409932374954224)]

# Problématique 
Discuter les rapports possibles entre les vecteurs des positions 

- Nom d'Action => production
- Nom d'Agent => producteur/productrice
- Adjectif relationnel => productif/productive
- autres dérivés (productivité, produit, produire, ...)

Est-ce qu'on peut trouver des généralisations sur les vecteurs différences entre ces positions ?
- un vecteur unique pour passer d'une position à une autre (pour qu'on arrive dans le voisinage de la position recherchée)
- un petit nombre de vecteurs pour passer d'une position à une autre

Si on a un petit nombre de vecteurs pour passer d'une position à une autre, est-ce qu'on peut trouver une généralisation pour prédire quel vecteur va marcher ?

## Faire une liste de paires à étudier
- nom d'action (ment-ion-age)/nom d'agent
    - lavage/laveur (eur-euse)
    - production/producteur (eur-rice)
    - accompagnement/accompagnateur (mélangé)
 
## Dérivations standard
- ment formé sur le thème du présent 6 (sulɛv+mâ)
- age formé sur le thème de l'imparfait (eləv+aZ)
- ion formé sur le thème du passé simple plus un t (programa+t+jô) il y a palatalisation du t [avec deux exceptions : gestion et question]

In [12]:
modelFr.most_similar(positive=["construction","producteur"],negative=["constructeur"],topn=5)

[('réalisation', 0.6321220993995667),
 ('production', 0.6101853847503662),
 ('danger1988', 0.5992995500564575),
 ('darbonne', 0.5985347032546997),
 ('ancienne1993', 0.5983116626739502)]

In [56]:
modelFr.most_similar(positive=["animant","destructeur"],negative=["animateur"],topn=5)

[('annihilant', 0.7579350471496582),
 ('répandant', 0.7097695469856262),
 ('détruisant', 0.7059866189956665),
 ('libérant', 0.7040006518363953),
 ('transformant', 0.6994748711585999)]

## Représentation des voisinages en 2D
TSNE permet une réduction de dimension pour visualiser les voisinages sur une carte

In [ ]:
def display_closestwords_tsnescatterplot(model, word, size,topn=10):
    
    arr = np.empty((0,size), dtype='f')
    word_labels = [word]
    close_words = model.similar_by_word(word,topn=topn)
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    plt.figure(figsize=(50,50))
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points',fontsize=50)
        plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
        plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()

In [ ]:
mot="roi"
display_closestwords_tsnescatterplot(modelFr, mot, 100,topn=100) 

### doesnt_match
doesnt_match cherche le vecteur le plus différent des autres dans une liste => chercher l'intrus

In [ ]:
modelFr.doesnt_match(['électricien','mécanicien','peintre','platrier','maçon','prof','voiture'])

In [ ]:
modelFr.doesnt_match("vélaire alvéolaire nasale dorsale vibrante petit".split(" "))

## Anglais Fasttext (NLPL/9)
Corpus :
- English Wikipedia February 2017
    - anglais, min/maj, lemmes, tagged
    - 2252637050 tokens

Créé par Andrey Kutuzov (andreku@ifi.uio.no)

Paramètres :
- Fasttext Skipgram
- 300 dimensions
- fenêtre 5 mots
- taille du vocabulaire : 273930 mots

In [2]:
path_to_modelEn = os.path.join(data_path,'9/parameters.bin')
#Load W2V model. This will take some time.
modelEn = gensim.models.fasttext.load_facebook_vectors(path_to_modelEn)
print('done loading Word2Vec')

done loading Word2Vec


### Modèles FastText
Les modèles FastText traitent non seulement les contextes des mots mais aussi ceux des N-grams. De ce fait, ils sont capables de fournir des vecteurs pour des mots qui ne sont pas dans le corpus du moment que les N-grams qui les composent sont effectivement dans le corpus.

In [27]:
modelEn.most_similar("table_NOUN")

[('table', 0.5758581161499023),
 ('table-tennis', 0.5082424879074097),
 ('tablebase', 0.5024933815002441),
 ('Carpianum', 0.5017015933990479),
 ('carrom', 0.48716774582862854),
 ('mini-league', 0.46465978026390076),
 ('Cup/Plate', 0.45730823278427124),
 ('four-ball', 0.4528191387653351),
 ('gameboard', 0.4524047374725342),
 ('Eusebian', 0.4486585557460785)]

In [32]:
modelEn.most_similar(positive=["king","woman"],negative=["man"])

[('monarch', 0.6749916076660156),
 ('princess', 0.638053297996521),
 ('regnant', 0.6341029405593872),
 ('queen', 0.6316593289375305),
 ('royal', 0.6289796829223633),
 ('kingdom', 0.5857125520706177),
 ('queen-consort', 0.5852508544921875),
 ('ruler', 0.5702740550041199),
 ('kingship', 0.5696046352386475),
 ('throne', 0.5673027634620667)]

In [37]:
modelEn.most_similar(positive=["spling","went"],negative=["go"])

[('splanchnic', 0.4732472002506256),
 ('derwent', 0.46673086285591125),
 ('Tawe', 0.4414108395576477),
 ('microcirculation', 0.440804123878479),
 ('fluid-filled', 0.4296455383300781),
 ('synovium', 0.42459219694137573),
 ('nasolacrimal', 0.4210854768753052),
 ('alimentary', 0.4143531918525696),
 ('utilisation', 0.4127790331840515),
 ('infilled', 0.41152435541152954)]

In [38]:
modelEn.most_similar("splang")

[('splanchnic', 0.6825659871101379),
 ('microcirculation', 0.6595008373260498),
 ('extravasation', 0.6080926656723022),
 ('vasculature', 0.6048488020896912),
 ('perfuse', 0.5823202133178711),
 ('endothelium', 0.5807481408119202),
 ('perivascular', 0.5788581371307373),
 ('nasolacrimal', 0.5781022906303406),
 ('radicular', 0.5777603983879089),
 ('pericardium', 0.5774173140525818)]

In [39]:
modelEn.doesnt_match("breakfast cereal field dinner lunch".split(" "))

/Users/gilles/opt/anaconda3/lib/python3.9/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'field'

## Représentation des voisinages en dendrogramme

In [ ]:
def makeDendrogram(model):
    l = linkage(model.vectors, method='complete', metric='seuclidean')

    # calculate full dendrogram
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.ylabel('word')
    plt.xlabel('distance')

    dendrogram(
        l,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=16.,  # font size for the x axis labels
        orientation='left',
        leaf_label_func=lambda v: str(model.wv.index2word[v])
    )
    plt.show()

In [ ]:
makeDendrogram(modelEn)

/var/folders/h7/0cygyb1j7cn88gdc53db7m6w0000gn/T/ipykernel_5801/170265316.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  l = linkage(model.wv.syn0, method='complete', metric='seuclidean')
/var/folders/h7/0cygyb1j7cn88gdc53db7m6w0000gn/T/ipykernel_5801/170265316.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  l = linkage(model.wv.syn0, method='complete', metric='seuclidean')


## Anglais
Corpus :
- British National Corpus
    - anglais, min/maj, lemmes, tagged
    - 1903181185 tokens

Créé par Andrey Kutuzov (andreku@ifi.uio.no)

Paramètres :
- Skipgram
- 300 dimensions
- fenêtre 10 mots
- taille du vocabulaire : 163473 mots 

In [ ]:
path_to_modelBNC = os.path.join(data_path,'0/model.bin')
#Load W2V model. This will take some time.
modelBNC = KeyedVectors.load_word2vec_format(path_to_modelBNC, binary=True)
print('done loading Word2Vec')

In [ ]:
modelBNC.most_similar("eat_VERB")

In [ ]:
modelBNC.most_similar(positive=["king_NOUN","woman_NOUN"],negative=["man_NOUN"],topn=5)

In [ ]:
modelBNC.doesnt_match("breakfast_NOUN cereal_NOUN dinner_NOUN lunch_NOUN".split(" "))